<a href="https://colab.research.google.com/github/KevinWang676/Bark-Voice-Cloning/blob/main/Sambert_Voice_Cloning_in_One_Click.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 全新中文声音克隆 Voice Cloning for Chinese Speech

## 配置环境 Set up

In [ ]:
! nvidia-smi # 需要使用GPU运行

Sat Sep  9 07:28:21 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    25W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!git clone https://huggingface.co/spaces/kevinwang676/Personal-TTS-v2

fatal: destination path 'Personal-TTS-v2' already exists and is not an empty directory.


In [ ]:
cd Personal-TTS-v2

/content/Personal-TTS-v2


In [ ]:
! pip install openai-whisper
! pip install modelscope
! pip install tts-autolabel -f https://modelscope.oss-cn-beijing.aliyuncs.com/releases/repo.html
! pip install typeguard==2.3.1
! pip install sox
! pip install bitstring
! pip install pysptk --no-build-isolation
! pip install kantts -f https://modelscope.oss-cn-beijing.aliyuncs.com/releases/repo.html
! pip install pytorch_wavelets
! pip install tensorboardX
! git clone https://github.com/fbcotter/pytorch_wavelets
! pip install matplotlib
! pip install numpy==1.22.0
! apt-get install sox

Looking in links: https://modelscope.oss-cn-beijing.aliyuncs.com/releases/repo.html
  Using cached numpy-1.23.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.0
    Uninstalling numpy-1.22.0:
      Successfully uninstalled numpy-1.22.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-gbq 0.17.9 requires pyarrow<10.0dev,>=3.0.0, but you have pyarrow 13.0.0 which is incompatible.


Looking in links: https://modelscope.oss-cn-beijing.aliyuncs.com/releases/repo.html
fatal: destination path 'pytorch_wavelets' already exists and is not an empty directory.
  Using cached numpy-1.22.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.1
    Uninstalling numpy-1.23.1:
      Successfully uninstalled numpy-1.23.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
librosa 0.10.1 requires numpy!=1.22.0,!=1.22.1,!=1.22.2,>=1.20.3, but you have numpy 1.22.0 which is incompatible.
pandas-gbq 0.17.9 requires pyarrow<10.0dev,>=3.0.0, but you have pyarrow 13.0.0 which is incompatible.
plotnine 0.12.3 requires numpy>=1.23.0, but you have numpy 1.22.0 which is incompatible.


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
sox is already the newest version (14.4.2+git20190427-2+deb11u2ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 16 not upgraded.


In [ ]:
cd pytorch_wavelets

/content/Personal-TTS-v2/pytorch_wavelets


In [ ]:
pip install .

Processing /content/Personal-TTS-v2/pytorch_wavelets
  Preparing metadata (setup.py) ... done
  Created wheel for pytorch-wavelets: filename=pytorch_wavelets-1.3.0-py3-none-any.whl size=54852 sha256=cd4232f63138e8a8d6e09752e8296074b3653966a7e03a90ef6c44dd7a69f88a
  Stored in directory: /tmp/pip-ephem-wheel-cache-hlkhgtfy/wheels/9a/f5/a7/b8c901126021c077bfd6fbe9f8100e26345c85e77e49f5d444
Successfully built pytorch-wavelets
  Attempting uninstall: pytorch-wavelets
    Found existing installation: pytorch-wavelets 1.3.0
    Uninstalling pytorch-wavelets-1.3.0:
      Successfully uninstalled pytorch-wavelets-1.3.0


In [ ]:
cd ..

/content/Personal-TTS-v2


In [ ]:
import sox
! pip install gradio
import gradio as gr

In [ ]:
import subprocess
import random
import os
from pathlib import Path
import librosa
from scipy.io import wavfile
import numpy as np
import torch
import csv
import whisper

def split_long_audio(model, filepaths, save_dir="data_dir", out_sr=44100):
    if isinstance(filepaths, str):
        filepaths = [filepaths]

    for file_idx, filepath in enumerate(filepaths):

        save_path = Path(save_dir)
        save_path.mkdir(exist_ok=True, parents=True)

        print(f"Transcribing file {file_idx}: '{filepath}' to segments...")
        result = model.transcribe(filepath, word_timestamps=True, task="transcribe", beam_size=5, best_of=5)
        segments = result['segments']

        wav, sr = librosa.load(filepath, sr=None, offset=0, duration=None, mono=True)
        wav, _ = librosa.effects.trim(wav, top_db=20)
        peak = np.abs(wav).max()
        if peak > 1.0:
            wav = 0.98 * wav / peak
        wav2 = librosa.resample(wav, orig_sr=sr, target_sr=out_sr)
        wav2 /= max(wav2.max(), -wav2.min())

        for i, seg in enumerate(segments):
            start_time = seg['start']
            end_time = seg['end']
            wav_seg = wav2[int(start_time * out_sr):int(end_time * out_sr)]
            wav_seg_name = f"{file_idx}_{i}.wav"
            out_fpath = save_path / wav_seg_name
            wavfile.write(out_fpath, rate=out_sr, data=(wav_seg * np.iinfo(np.int16).max).astype(np.int16))

whisper_size = "medium"
whisper_model = whisper.load_model(whisper_size)

In [ ]:
from modelscope.tools import run_auto_label

from modelscope.models.audio.tts import SambertHifigan
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks

from modelscope.metainfo import Trainers
from modelscope.trainers import build_trainer
from modelscope.utils.audio.audio_utils import TtsTrainType

pretrained_model_id = 'damo/speech_personal_sambert-hifigan_nsf_tts_zh-cn_pretrain_16k'

dataset_id = "/content/Personal-TTS-v2/output_training_data/"
pretrain_work_dir = "/content/Personal-TTS-v2/pretrain_work_dir/"


def auto_label(audio):
    try:
        split_long_audio(whisper_model, audio, "/content/Personal-TTS-v2/test_wavs")
        input_wav = "/content/Personal-TTS-v2/test_wavs/"
        output_data = "/content/Personal-TTS-v2/output_training_data/"
        ret, report = run_auto_label(input_wav=input_wav, work_dir=output_data, resource_revision="v1.0.7")

    except Exception:
        pass
    return "标注成功"


def train(a):
    try:
        train_info = {
            TtsTrainType.TRAIN_TYPE_SAMBERT: {  # 配置训练AM（sambert）模型
                'train_steps': 52,               # 训练多少个step
                'save_interval_steps': 50,       # 每训练多少个step保存一次checkpoint
                'log_interval': 10               # 每训练多少个step打印一次训练日志
            }
        }

        # 配置训练参数，指定数据集，临时工作目录和train_info
        kwargs = dict(
            model=pretrained_model_id,                  # 指定要finetune的模型
            model_revision = "v1.0.6",
            work_dir=pretrain_work_dir,                 # 指定临时工作目录
            train_dataset=dataset_id,                   # 指定数据集id
            train_type=train_info                       # 指定要训练类型及参数
        )

        trainer = build_trainer(Trainers.speech_kantts_trainer,
                            default_args=kwargs)

        trainer.train()

    except Exception:
        pass

    return "训练完成"


import random

def infer(text):

  model_dir = "/content/Personal-TTS-v2/pretrain_work_dir"

  custom_infer_abs = {
      'voice_name':
      'F7',
      'am_ckpt':
      os.path.join(model_dir, 'tmp_am', 'ckpt'),
      'am_config':
      os.path.join(model_dir, 'tmp_am', 'config.yaml'),
      'voc_ckpt':
      os.path.join(model_dir, 'orig_model', 'basemodel_16k', 'hifigan', 'ckpt'),
      'voc_config':
      os.path.join(model_dir, 'orig_model', 'basemodel_16k', 'hifigan',
              'config.yaml'),
      'audio_config':
      os.path.join(model_dir, 'data', 'audio_config.yaml'),
      'se_file':
      os.path.join(model_dir, 'data', 'se', 'se.npy')
  }
  kwargs = {'custom_ckpt': custom_infer_abs}

  model_id = SambertHifigan(os.path.join(model_dir, "orig_model"), **kwargs)

  inference = pipeline(task=Tasks.text_to_speech, model=model_id)
  output = inference(input=text)

  filename = str(random.randint(1, 1000000000000))

  with open(filename + "myfile.wav", mode='bx') as f:
      f.write(output["output_wav"])
  return filename + "myfile.wav"

2023-09-09 07:29:57,174 - modelscope - INFO - PyTorch version 2.0.1+cu118 Found.
2023-09-09 07:29:57,178 - modelscope - INFO - TensorFlow version 2.13.0 Found.
2023-09-09 07:29:57,179 - modelscope - INFO - Loading ast index from /root/.cache/modelscope/ast_indexer
2023-09-09 07:29:57,214 - modelscope - INFO - Loading done! Current index file version is 1.9.0, with md5 26bab0b425de9d3ec01e3de23e7c0f4d and a total number of 921 components indexed


In [ ]:
auto_label("nana_speech.wav")
train("test")
infer("欢迎使用滔滔智能的声音克隆产品")

Transcribing file 0: 'nana_speech.wav' to segments...


2023-09-09 07:30:22,206 - modelscope - INFO - Use user-specified model revision: v1.0.7


2023-09-09 07:30:30
TTS-AutoLabel version: 1.1.8
TTS-AutoLabel resource path: /root/.cache/modelscope/hub/damo/speech_ptts_autolabel_16k/model
Target sampling rate: 16000
Input wav dir: /content/Personal-TTS-v2/test_wavs
Output data dir: /content/Personal-TTS-v2/output_training_data
wav_preprocess start...
---  new folder...  ---
---  OK  ---


100%|██████████| 34/34 [00:00<00:00, 112.07it/s]

[VAD] chunk recordings for training.
wav cut by vad start...


...


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 51 finished



2023-09-09 07:32:35,661 - modelscope - INFO - SAMBERT training spent: 0.01 hours

2023-09-09 07:32:35,662 - modelscope - INFO - skip HIFIGAN training...
2023-09-09 07:32:35,814 - modelscope - INFO - am_config=/content/Personal-TTS-v2/pretrain_work_dir/tmp_am/config.yaml voc_config=/content/Personal-TTS-v2/pretrain_work_dir/orig_model/basemodel_16k/hifigan/config.yaml
2023-09-09 07:32:35,815 - modelscope - INFO - audio_config=/content/Personal-TTS-v2/pretrain_work_dir/data/audio_config.yaml
2023-09-09 07:32:35,816 - modelscope - INFO - am_ckpts=OrderedDict([(2400000, '/content/Personal-TTS-v2/pretrain_work_dir/tmp_am/ckpt/checkpoint_2400000.pth'), (2400050, '/content/Personal-TTS-v2/pretrain_work_dir/tmp_am/ckpt/checkpoint_2400050.pth')])
2023-09-09 07:32:35,818 - modelscope - INFO - voc_ckpts=OrderedDict([(2400000, '/content/Personal-TTS-v2/pretrain_work_dir/orig_model/basemodel_16k/hifigan/ckpt/checkpoint_2400000.pth')])
2023-09-09 07:32:35,819 - modelscope - INFO - se_path=/content/

Removing weight norm...


'69792480141myfile.wav'

In [ ]:
app = gr.Blocks()

with app:
    gr.Markdown("# <center>🥳🎶🎡 - Sambert中文声音克隆</center>")
    gr.Markdown("## <center>🌟 - 训练3分钟，推理5秒钟，中英自然发音 </center>")
    gr.Markdown("### <center>🌊 - 更多精彩应用，敬请关注[滔滔AI](http://www.talktalkai.com)；滔滔AI，为爱滔滔！💕</center>")

    with gr.Row():
      inp1 = gr.Audio(type="filepath", label="请上传一段音频")
      out1 = gr.Textbox(label="标注情况", lines=1, interactive=False)

      out2 = gr.Textbox(label="训练情况", lines=1, interactive=False)
      inp2 = gr.Textbox(label="文本", lines=3)
      out3 = gr.Audio(type="filepath", label="合成的音频")
      btn1 = gr.Button("1.标注数据")
      btn2 = gr.Button("2.开始训练")
      btn3 = gr.Button("3.一键推理", variant="primary")

      btn1.click(auto_label, inp1, out1)
      btn2.click(train, out1, out2)
      btn3.click(infer, inp2, out3)

    gr.Markdown("### <center>注意❗：请不要生成会对个人以及组织造成侵害的内容，此程序仅供科研、学习及个人娱乐使用。</center>")
    gr.HTML('''
        <div class="footer">
                    <p>🌊🏞️🎶 - 江水东流急，滔滔无尽声。 明·顾璘
                    </p>
        </div>
    ''')
app.launch(show_error=True, share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://c20364d25b1112c758.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
